In [3]:
from pynq import Overlay
from pynq import MMIO
import time
design = Overlay("./Final_v5/cnn.bit")

cdma_address = design.ip_dict['axi_cdma_0']['phys_addr']
axi_gpio_address0 = design.ip_dict['axi_gpio_0']['phys_addr'] # start
axi_gpio_address1 = design.ip_dict['axi_gpio_1']['phys_addr'] # done
axi_gpio_address2 = design.ip_dict['axi_gpio_2']['phys_addr'] # mode
axi_gpio_address3 = design.ip_dict['axi_gpio_3']['phys_addr'] # result

zynq_addr = 0x30000000


bram0_addr = 0xC0000000 # if1
bram1_addr = 0xC2000000 # if2
bram2_addr = 0xC4000000 # w1
bram3_addr = 0xC6000000 # w2
bram4_addr = 0xC8000000 # w3
bram5_addr = 0xCA000000 # w4
bram6_addr = 0xCC000000 # w5

zynq_sys = MMIO(zynq_addr,    0xC000)
cdma     = MMIO(cdma_address, 0xC000)

'''

initial input file to bram0
    
    
'''
print('start initial!\n')
print('Input feature map initial!\n')
bram0_offset = 0x0

with open("number_conv1_32_in.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram0_offset,int('0x'+line1,16))      
        bram0_offset += 4
       
cdma.write(0x00,0x04) #start
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram0_addr)
cdma.write(0x28,0x400) # 1024



'''

initial conv1 weight to bram2
    
    
'''
print('Weight1 initial!\n')

bram2_offset = 0x0

with open("number_conv1_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram2_offset,int('0x'+line1,16))      
        bram2_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram2_addr)
cdma.write(0x28,0x98) # 152 = 38 * 4

'''

initial conv2 weight to bram3
    
    
'''
print('Weight2 initial!\n')

bram3_offset = 0x0

with open("number_conv2_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram3_offset,int('0x'+line1,16))      
        bram3_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram3_addr)
cdma.write(0x28,0x960) # 2400 = 600 * 4


'''

initial conv3 weight to bram4
    
    
'''
print('Weight3 initial!\n')

bram4_offset = 0x0

with open("number_conv3_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram4_offset,int('0x'+line1,16))      
        bram4_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram4_addr)
cdma.write(0x28,0xBB80) # 48000 = 12000 * 4

'''

initial fc1 weight to bram5
    
    
'''
print('Weight4 initial!\n')

bram5_offset = 0x0

with open("number_fc1_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram5_offset,int('0x'+line1,16))      
        bram5_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram5_addr)
cdma.write(0x28,0x2760) # 10080 = 2520 * 4

'''

initial fc2 weight to bram6
    
    
'''
print('Weight5 initial!\n')

bram6_offset = 0x0

with open("number_fc2_32.hex", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram6_offset,int('0x'+line1,16))      
        bram6_offset += 4

cdma.write(0x00,0x04)
cdma.write(0x18,zynq_addr)
cdma.write(0x20,bram6_addr)
cdma.write(0x28,0x348) # 840 = 210 * 4

print('initial finish!\n')


'''

Input start signal
 
''' 

gpio_a = MMIO(axi_gpio_address0,8)
gpio_b = MMIO(axi_gpio_address1,8)
gpio_c = MMIO(axi_gpio_address2,8)
gpio_d = MMIO(axi_gpio_address3,8)
print("done = ", hex(gpio_b.read()))
gpio_a.write(0x4,0)
gpio_c.write(0x4,0)
gpio_c.write(0,1) # write 1: number
start_time = time.time()
gpio_a.write(0,1) # write 1 to start

         
gpio_a.write(0,0)
    
'''    
bram3_offset = 0x0

cdma.write(0x0,0x04)
cdma.write(0x18,bram0_addr)
cdma.write(0x20,zynq_addr)
cdma.write(0x28,0x190) # conv1: 498 = 14 * 14 * 6


print("done = ", hex(gpio_b.read()))
with open('cnn_output_number.hex', 'w') as f_out:
    while bram3_offset<100*4:   # conv1: 294
        ans = str(hex(zynq_sys.read(bram3_offset)))[2:]
        if len(ans)!=8:    # 補 0 
            ans = '0'*(8-len(ans))+ans
        f_out.write(ans+'\n')
        bram3_offset += 4
'''


#while gpio_b.read() != 0xffffffff:
print("--- %s seconds ---" % (time.time() - start_time))
print("Inference value: ", gpio_d.read())
print('all finish!')

start initial!

Input feature map initial!

Weight1 initial!

Weight2 initial!

Weight3 initial!

Weight4 initial!

Weight5 initial!

initial finish!

done =  0x0
--- 0.0012292861938476562 seconds ---
Inference value:  7
all finish!
